# Lab 3.4.5: Multi-Agent System - Solutions

This notebook contains complete solutions for the Multi-Agent exercises.

---

## Challenge: Extended Agent Team

**Solution:** Adding Fact Checker, SEO Specialist, and Translator agents.

In [ ]:
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional
from datetime import datetime
from langchain_community.llms import Ollama

# Initialize LLM
llm = Ollama(model="llama3.1:8b", temperature=0.7)

@dataclass
class Agent:
    """Base agent class."""
    name: str
    role: str
    personality: str
    llm: Any
    
    def execute(self, task: str, context: str = "") -> str:
        prompt = f"""You are {self.name}, a {self.role}.
Personality: {self.personality}

Context:
{context}

Task:
{task}

Your response:"""
        print(f"🤖 {self.name} working...")
        return self.llm.invoke(prompt)

print("Agent class ready!")

In [ ]:
# Define the extended team

# Original team
researcher = Agent(
    name="Alex",
    role="Research Specialist",
    personality="Thorough, detail-oriented, finds interesting facts.",
    llm=llm
)

writer = Agent(
    name="Sam",
    role="Content Writer",
    personality="Creative, engaging, turns facts into stories.",
    llm=llm
)

editor = Agent(
    name="Jordan",
    role="Editorial Reviewer",
    personality="Meticulous, improves clarity and flow.",
    llm=llm
)

# NEW: Additional specialists
fact_checker = Agent(
    name="Dana",
    role="Fact Checker",
    personality="""Skeptical but fair. Verifies claims against known facts.
Flags anything uncertain with [VERIFY]. Approves accurate content.""",
    llm=llm
)

seo_specialist = Agent(
    name="Chris",
    role="SEO Specialist",
    personality="""Keyword-savvy, understands search algorithms.
Suggests title improvements, meta descriptions, and keyword optimization.""",
    llm=llm
)

translator = Agent(
    name="Mia",
    role="Multilingual Translator",
    personality="""Fluent in multiple languages. Preserves tone and meaning.
Adapts cultural references appropriately.""",
    llm=llm
)

print("Extended team created:")
for agent in [researcher, writer, editor, fact_checker, seo_specialist, translator]:
    print(f"  - {agent.name}: {agent.role}")

In [ ]:
@dataclass
class ExtendedContentTask:
    """Extended content task with all stages."""
    topic: str
    content_type: str
    target_audience: str
    target_languages: List[str] = field(default_factory=list)
    
    # Stage outputs
    research: str = ""
    draft: str = ""
    edited_draft: str = ""
    fact_check_report: str = ""
    seo_optimized: str = ""
    translations: Dict[str, str] = field(default_factory=dict)
    
    final_content: str = ""

print("ExtendedContentTask defined!")

In [ ]:
class ExtendedOrchestrator:
    """Orchestrator for the extended team."""
    
    def __init__(self):
        self.researcher = researcher
        self.writer = writer
        self.editor = editor
        self.fact_checker = fact_checker
        self.seo_specialist = seo_specialist
        self.translator = translator
    
    def create_content(self, task: ExtendedContentTask) -> ExtendedContentTask:
        """Run the full extended pipeline."""
        print(f"\n{'='*60}")
        print(f"Creating: {task.content_type} about '{task.topic}'")
        print(f"{'='*60}")
        
        # Stage 1: Research
        print("\n--- Stage 1: Research ---")
        task.research = self.researcher.execute(
            f"Research: {task.topic}. Target audience: {task.target_audience}."
        )
        
        # Stage 2: Write
        print("\n--- Stage 2: Writing ---")
        task.draft = self.writer.execute(
            f"Write a {task.content_type} about {task.topic}.",
            context=f"Research:\n{task.research}"
        )
        
        # Stage 3: Edit
        print("\n--- Stage 3: Editing ---")
        task.edited_draft = self.editor.execute(
            f"Edit and improve this {task.content_type}.",
            context=f"Draft:\n{task.draft}"
        )
        
        # Stage 4: Fact Check
        print("\n--- Stage 4: Fact Checking ---")
        task.fact_check_report = self.fact_checker.execute(
            "Verify all claims and facts in this content. Flag any issues.",
            context=f"Content:\n{task.edited_draft}"
        )
        
        # Stage 5: SEO Optimization
        print("\n--- Stage 5: SEO Optimization ---")
        task.seo_optimized = self.seo_specialist.execute(
            f"""Optimize this {task.content_type} for search engines.
Suggest: title, meta description, keywords, and any content improvements.""",
            context=f"Content:\n{task.edited_draft}"
        )
        
        # Stage 6: Translation (if requested)
        if task.target_languages:
            print("\n--- Stage 6: Translation ---")
            for lang in task.target_languages:
                print(f"  Translating to {lang}...")
                task.translations[lang] = self.translator.execute(
                    f"Translate this to {lang}. Preserve tone and meaning.",
                    context=f"Content:\n{task.edited_draft}"
                )
        
        task.final_content = task.edited_draft
        print("\n✅ All stages complete!")
        
        return task

orchestrator = ExtendedOrchestrator()
print("Extended orchestrator ready!")

In [ ]:
# Create and run an extended content task
task = ExtendedContentTask(
    topic="How AI is Transforming Healthcare",
    content_type="blog post",
    target_audience="Healthcare professionals and tech enthusiasts",
    target_languages=["Spanish"]  # Request translation
)

result = orchestrator.create_content(task)

In [ ]:
# View all outputs
print("\n" + "="*60)
print("FINAL CONTENT")
print("="*60)
print(result.final_content)

print("\n" + "="*60)
print("FACT CHECK REPORT")
print("="*60)
print(result.fact_check_report)

print("\n" + "="*60)
print("SEO RECOMMENDATIONS")
print("="*60)
print(result.seo_optimized)

if result.translations:
    for lang, content in result.translations.items():
        print(f"\n{'='*60}")
        print(f"TRANSLATION ({lang.upper()})")
        print("="*60)
        print(content[:500] + "...")

## Key Takeaways

1. **Specialized agents** allow deeper expertise in each area
2. **Sequential pipeline** ensures quality gates at each stage
3. **Fact checking** catches errors before publication
4. **SEO optimization** improves content discoverability
5. **Translation** enables global reach with consistent quality